In [1]:
import phe   # installed via pip
import numpy as np

def generate_vct(D=100, NZ=10):
    v = np.random.randint(0, 10, NZ)
    K = np.random.permutation(D)[:NZ]
    w = np.zeros(D).astype('int')
    w[K] = v
    return w, v, K

def dphe_genkey(keylen=128, D=100, NU=3):
    enc, dec = phe.generate_paillier_keypair(n_length=keylen)
    E0 = enc.encrypt(0)
    Pi = [np.random.permutation(D) for _ in range(NU)]
    P = np.random.permutation(D)
    return enc, dec, E0, Pi, P

def dphe_enc(v, K, enc, Pi, P):
    e = [enc.encrypt(v_) for v_ in v]
    DK = Pi[P[K]]    
    return e, DK 

def dphe_recon(e, DK, Pi, E0):
    SK = np.argsort(Pi)[DK]
    SEw = np.hstack([E0 for _ in range(len(Pi))])
    for i in range(len(SK)):
        SEw[SK[i]] = e[i]
    return SEw

def dphe_dec(SEw, dec, P):
    Sw = np.hstack([dec.decrypt(ew_) for ew_ in SEw])
    w = Sw[P]
    
    return w

def baseline_enc(w, enc):
    ew = np.hstack([enc.encrypt(w_) for w_ in w])
    return ew

def baseline_dec(ew, dec):
    w = np.hstack([dec.decrypt(ew_) for ew_ in ew])
    return w

Let's begin by a simple secure-sum problem. Users 1, 2, 3 have a secret value $a_1, a_2, a_3$, respectively. An aggregator wishes to compute the sum of these values, $\sum_i a_i$, while keeping each $a_i$ secret.

In [2]:
a1 = 1
a2 = 2
a3 = 3
print 'a1:', a1, 'a2:', a2, 'a3:', a3

a1: 1 a2: 2 a3: 3


To solve this problem, we use Paillier homomorphic cryptosystem. First, a certain key generator issues both encryption and decryption keys. He then distributes the encryption key to the parties.

In [3]:
# Key generator side
enc, dec = phe.generate_paillier_keypair(n_length=1024)

Users encrypt their secret value with the given key and send the ciphertext to the aggregator. Since the aggregator does not have a decryption key, he cannot identify plaintext $a_i$ from the ciphertext $E(a_i)$.

In [4]:
# User side

e1 = enc.encrypt(a1)
e2 = enc.encrypt(a2)
e3 = enc.encrypt(a3)
print 'E(a1):', e1.ciphertext()
print 'E(a2):', e2.ciphertext()
print 'E(a3):', e3.ciphertext()

E(a1): 9818165258279516697981057867858941026201112128676847046501819099837080152386082662253067759962086104407916371785728394643155714388067505352951175484095036617696628082547114146442942644763113227275261352331713264124776033453373973525891038708718073912852842699360397251817771559231584108735949759034869841782564526304806441167391910022053125077100308911802626677450559353227890997019220616611975794700019508836218080578266058102145707355929978725547685437405960831752205328795567997038791620459743630461558411361861716833202776169670327174803692207298696389814524381670273500226373266134917391256374080000970104456012
E(a2): 730615143021740765212816929275916577116012575863371809918713859234859590337355437882571450431727225776565474614982742012883102162413293979141252955258709135159798794088513396690401583789032861600240607026099626071371735879534866582891691535399603511402376763630735164885701620762340392595884520742362893376179321949756690793804821284657351602796124354003681172069453892

With that said, the aggregator is still able to compute the encrypted sum of plaintexts: $E(\sum a_i)= \prod E(a_i)$

In [5]:
# Aggregator side
e_sum = e1 + e2 + e3
print 'E(\sum ai):', e_sum.ciphertext()

E(\sum ai): 5740316976987220801254426437017751356156962861712774716232866779664686187337884696195935314626663663264195963729080253856478992121687305958632136922519939260595393228901264725902342620355583206789643010957909747689398060656864916849632669074987283460797299400892309666585125267745152069580024620465478694272854710555235646900552736719441632675473772811542296529736560389312580572538810407430911753527168342289528848507976769212013006441549335342876172008324123487832488639320536528619547566480633511040666129271238854495845930081662846476838704647597711976254166709011443635578141464179372094004642812472219520547657


Finally, the aggregator asks the key generator to decrypt $E(\sum a_i)$. Although the aggregator and the key generator have $\sum a_i$, they cannot identify each $a_i$ from this information.

In [6]:
# Key generator side
a_sum = dec.decrypt(e_sum)
print '\sum ai:', a_sum

\sum ai: 6


Let's go on a more general case. Now, each user has a high-dimensional sparse vector.

In [7]:
keylen = 1024
D = 200
NZ = 20
NU = 3
w1, v1, K1 = generate_vct(D, NZ)
w2, v2, K2 = generate_vct(D, NZ)
w3, v3, K3 = generate_vct(D, NZ)
print 'w1:', w1
print 'w2:', w2
print 'w3:', w3

w1: [0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 8 0 0 0 0 0 0 0 6 2 0 0 0 0 0 0 0 0 0 0 0 0
 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0
 0 0 5 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 1 0 0
 0 0 0 0 0 9 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
w2: [0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9 0 0 9 0 0 2 0 0 0 0 0 0 0 0 4
 4 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 5 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0
 0 0 0 0 0 0 0 6 0 0 0 0 0 0 9]
w3: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 7 0 0 0 0 0 6
 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 3 0 0 0 0 0 0 0 0

The straightforward extension of the previous secure-sum algorithm to accept vector inputs is to apply Paillier encryption element-wise. However, this takes a lot of encryption time.

In [8]:
%%time

## User side
ew1 = baseline_enc(w1, enc)
ew2 = baseline_enc(w2, enc)
ew3 = baseline_enc(w3, enc)

CPU times: user 2.18 s, sys: 28 ms, total: 2.2 s
Wall time: 2.17 s


In [9]:
# Aggregator side
sum_ew = ew1 + ew2 + ew3

# Key generator side
sum_w = baseline_dec(sum_ew, dec)
print '\sum w:', sum_w, 'diff:', np.linalg.norm(sum_w - (w1 + w2 + w3))

\sum w: [ 0  0  0  0  0  0  3  0  0  0  1  5  0  0  0  8  0  0  0  0  0 10  0  6 11
  0  0  2  0  0  7  0  0  0  0  0 10  8  0  0  0  4  9  0  0  0  0  0  0  0
  0  0  4  0  0  0  5  0  3  6  0  0  0  4  0  0  0  0  0  0  0  0  0  0  5
  0  0  7  0  0  0  0  0  0  9  0  0  0  4  0  0  1  7  0  0  0  0  0  0  0
  6  0  5  1  0  5  0  0  0  0  0  0  0  5  2  0  0  2  3  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0  1  6  0  0  0
  0  0  0  9  0  0  0  0  9  0  0  0  0  0  0  0  2  9  0  0  0  0  3  0  0
  0  0  0  0  0 10  3  0  0  0  4  3  0  0  0  0  0  6  0  0  0  0  0  0  9] diff: 0.0


Our DPHE is an efficient encryption scheme tailored to such high-dimensional sparse data. We apply the Paillier encryption only to non-zero values of a vector. Non-zero indices are shuffled with two different permutation matrices: 1) User-Aggregator matrix $P_i$ shared only between $i$-th user and the aggregator, and 2) User-shared matrix $P$ shared among all users but the aggregator. These two matrices are generated together with Paillier keys by a key generator.

In [10]:
# Key generator side
enc, dec, E0, Pi, P = dphe_genkey(keylen, D, NU)

In [11]:
%%time

# User side
ev1, DK1 = dphe_enc(v1, K1, enc, Pi[0], P)
ev2, DK2 = dphe_enc(v2, K2, enc, Pi[1], P)
ev3, DK3 = dphe_enc(v3, K3, enc, Pi[2], P)

CPU times: user 284 ms, sys: 8 ms, total: 292 ms
Wall time: 290 ms


As the aggregator has the only $E(v_i)$ (encryption of non-zero values), $P_iPK_i$ (doubly-permuted non-zero indices, and $P_i$, he cannot identify neither $v_i$ nor $K_i$. However, he is still able to compute $PE(w_i)$ and thus $PE(\sum w_i)$. Finally, the key generator decrypt $PE(\sum w_i)$ to share $\sum w_i$.

In [12]:
# Aggregator side
sew1 = dphe_recon(ev1, DK1, Pi[0], E0)
sew2 = dphe_recon(ev2, DK2, Pi[1], E0)
sew3 = dphe_recon(ev3, DK3, Pi[2], E0)
sum_sew = sew1 + sew2 + sew3

# Key generator side
sum_w = dphe_dec(sum_sew, dec, P)
print sum_w, np.linalg.norm(sum_w - (w1 + w2 + w3))

[ 0  0  0  0  0  0  3  0  0  0  1  5  0  0  0  8  0  0  0  0  0 10  0  6 11
  0  0  2  0  0  7  0  0  0  0  0 10  8  0  0  0  4  9  0  0  0  0  0  0  0
  0  0  4  0  0  0  5  0  3  6  0  0  0  4  0  0  0  0  0  0  0  0  0  0  5
  0  0  7  0  0  0  0  0  0  9  0  0  0  4  0  0  1  7  0  0  0  0  0  0  0
  6  0  5  1  0  5  0  0  0  0  0  0  0  5  2  0  0  2  3  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0  1  6  0  0  0
  0  0  0  9  0  0  0  0  9  0  0  0  0  0  0  0  2  9  0  0  0  0  3  0  0
  0  0  0  0  0 10  3  0  0  0  4  3  0  0  0  0  0  6  0  0  0  0  0  0  9] 0.0
